## Imports

In [13]:
import pandas as pd
import numpy as np
import re
import os
import folium
import geopandas as gpd
import glob
from datetime import datetime, timedelta

## Data Read-in

In [64]:
csv_files = glob.glob('*.csv')

df_list = []

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    df_list.append(df)
    
df = pd.concat(df_list,ignore_index=True)

print(len(df))

2926


In [65]:
gdf = gpd.read_file('Miami-Dade_Boundary.geojson')

## Previous month's values

Get values from here:

https://docs.google.com/spreadsheets/d/1GWlqTIzCB29pDVDjd-iBVejVGGsLiAd0P6fq4LM83WI/edit?usp=sharing

In [ ]:
total_condos_sold_2nd_month = ''
sales_volume_2nd_month = ''
median_sales_price_2nd_month = ''
median_psf_2nd_month = ''

## Data Clean

In [66]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [67]:
df = df.dropna(subset=['SOLD DATE'])

In [120]:
# Get the current date
current_date = datetime.now()

# Calculate the first day of the current month
first_day_of_current_month = current_date.replace(day=1)

# Get the last day of the previous month by subtracting one day from the first day of the current month
last_day_of_previous_month = first_day_of_current_month - timedelta(days=1)

# Get the month name of the previous month
previous_month_name = last_day_of_previous_month.strftime("%B")

# ---- Get the name of the month, as a string, from two months ago.
# ---- Meaning, if the current month is January, return November.

# Get the first day of the previous month by subtracting one day from the first day of the current month
first_day_of_previous_month = first_day_of_current_month - timedelta(days=1)

# Get the first day of the month before the last by subtracting one day from the first day of the previous month
first_day_of_month_before_last = first_day_of_previous_month.replace(day=1) - timedelta(days=1)

# Get the month name of the month before the last
month_before_last_name = first_day_of_month_before_last.strftime("%B")

In [69]:
# Define list of desired months (excluding current month)
desired_months = [previous_month_name]

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [70]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    739
Name: count, dtype: int64
-------
$/SQUARE FEET
False    735
True       4
Name: count, dtype: int64
-------
YEAR BUILT
False    739
Name: count, dtype: int64
-------


In [71]:
sorted_df = df_filtered.sort_values(by='YEAR BUILT', ascending=False)
second_newest_building = sorted_df.iloc[2]
print(second_newest_building['URL'])

https://www.redfin.com/FL/Miami-Beach/6800-Indian-Creek-Dr-33141/unit-10A/home/185381541


In [72]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [73]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [74]:
df_filtered = df_filtered.drop_duplicates().reset_index(drop=True)

In [75]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
507,PAST SALE,December-12-2023,Condo/Co-op,6970 NW 186th St Unit 3-502,Hialeah,FL,33015.0,260.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Gardens/6970-N...,MARMLS,A11462654,N,Y,25.939827,-80.315895
517,PAST SALE,December-6-2023,Condo/Co-op,7975 Crespi Blvd #2,Miami Beach,FL,33141.0,28000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7975-Cre...,MARMLS,A11387790,N,Y,25.865374,-80.126150
505,PAST SALE,December-15-2023,Condo/Co-op,6450 Collins Ave #1107,Miami Beach,FL,33141.0,44000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6450-Col...,MARMLS,A11454319,N,Y,25.847518,-80.120504
499,PAST SALE,December-4-2023,Condo/Co-op,251 NW 177th St Unit A-121,Miami Gardens,FL,33169.0,75000.0,0.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Gardens/251-NW...,Beaches MLS,F10400138,N,Y,25.937450,-80.206349
513,PAST SALE,December-11-2023,Condo/Co-op,52 NE 204th St #25,Miami Gardens,FL,33179.0,115000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Gardens/52-NE-...,Beaches MLS,F10389098,N,Y,25.962917,-80.200185
525,PAST SALE,December-15-2023,Condo/Co-op,2025 NE 164th St #807,North Miami Beach,FL,33162.0,120000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/20...,MARMLS,A11365841,N,Y,25.927607,-80.159386
487,PAST SALE,December-19-2023,Condo/Co-op,1590 NE 127th St #205,North Miami,FL,33161.0,125000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/1590-NE-...,MARMLS,A11463091,N,Y,25.892652,-80.166976
511,PAST SALE,December-19-2023,Condo/Co-op,2980 Point East Dr Unit D112,Aventura,FL,33160.0,132180.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/2980-Point-...,MARMLS,A11425718,N,Y,25.940986,-80.140743
520,PAST SALE,December-15-2023,Condo/Co-op,6541 NW 12th Ave #3,Miami,FL,33150.0,132900.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/6541-NW-12th-A...,MARMLS,A11401733,N,Y,25.834858,-80.216493
509,PAST SALE,December-19-2023,Condo/Co-op,1770 NE 191st St Unit 215-1,Miami,FL,33179.0,140000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/17...,MARMLS,A11462701,N,Y,25.951003,-80.166485


In [80]:
print(df_filtered['URL'].iloc[499])

https://www.redfin.com/FL/Miami-Gardens/251-NW-177th-St-33169/unit-A-121/home/42993968


In [81]:
# Correct the prices, if needed
df_filtered.at[507,'PRICE']=(260000)
df_filtered.at[517,'PRICE']=(280000)
df_filtered.at[505,'PRICE']=(440000)

In [ ]:
### If needed, drop the row with the lowest price
# min_value_index = df_filtered['PRICE'].idxmin()
# df_filtered = df_filtered.drop(min_value_index)

In [82]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE


In [87]:
# # Corrections, if needed
df_filtered.at[507,'$/SQUARE FEET']=(260000/907)
df_filtered.at[517,'$/SQUARE FEET']=(280000/657)
df_filtered.at[505,'$/SQUARE FEET']=(440000/1006)

In [90]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
508,149900.0,1770 NE 191st St Unit 711-1,Miami,114.0
509,140000.0,1770 NE 191st St Unit 215-1,Miami,132.0
500,178000.0,1401 NE Miami Gardens Dr #296,Miami,135.0
116,222000.0,15143 NE 6th Ave #5,Miami,137.0
519,155000.0,2025 NE 164th St #516,North Miami Beach,139.0
525,120000.0,2025 NE 164th St #807,North Miami Beach,152.0
487,125000.0,1590 NE 127th St #205,North Miami,156.0
513,115000.0,52 NE 204th St #25,Miami Gardens,162.0
537,276000.0,6821 SW 147th Ave Unit 2B,Miami,166.0
499,75000.0,251 NW 177th St Unit A-121,Miami Gardens,170.0


In [89]:
print(df_filtered.URL.iloc[116])

https://www.redfin.com/FL/Miami/15143-NE-6th-Ave-33162/unit-5/home/43014758


In [18]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [91]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [92]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [93]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [94]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [95]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [96]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"{previous_month_name} 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)
        
folium.GeoJson(gdf,tooltip='Miami-Dade County',name='Miami-Dade County').add_to(m)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map
m

In [97]:
m.save('index.html')

## Summary Info

In [98]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [99]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [100]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [142]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

https://www.redfin.com/FL/Miami-Beach/918-Ocean-Dr-33139/unit-205/home/42805206


In [102]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
PORSCHE DESIGN TOWER, 18555 Collins Ave #5201 Sunny Isles Beach | Price $10,500,000 | $1,715 psf | Year built: 2016
Least Expensive
New World Condo Apts Iii, 251 NW 177th St Unit A-121 Miami Gardens | Price $75,000 | $170 psf | Year built: 1971
Highest Price Per Square Foot
57 OCEAN CONDO, 5775 Collins Ave #303 Miami Beach | Price $8,765,000 | $3,019 psf | Year built: 2021
Lowest Price Per Square Foot
Jade Winds Group -, 1770 NE 191st St Unit 711-1 Miami | Price $149,900 | $114 psf | Year built: 1969
Newest
Missoni Baia, 700 NE 26th Ter #1703 Miami | Price $1,880,000 | $984 psf | Year built: 2023
Oldest
918 OCEAN DRIVE CONDO, 918 Ocean Dr #205 Miami Beach | Price $385,000 | $1,100 psf | Year built: 1925


## Time on Market Calculator

In [31]:
# print(df_Current.loc[df_Current['YEAR BUILT'].idxmin()])

In [138]:
from datetime import datetime, timedelta

date1 = datetime(2023, 11, 7) ## List (Earlier) date
date2 = datetime(2023, 12, 4) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

27


## Map URL Snagger

In [130]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [131]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MD_condo_sales_month_ending_dec_2023


## Get Summary Data

In [104]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 677
--------
Total sale price: $456,538,943
--------
Median sale price: $394,000
--------
Max sale price: $10,500,000
--------
Min sale price: $75,000
------------------------------------------------
PSF INFO
Max price per square foot: $3,019
--------
Min price per square foot: $114
--------
Median price per square foot: $426
------------------------------------------------
CONDO AGES
Newest building: 2023.0
----------
Oldest building: 1925.0
----------
Average building age: 1988.327917282127


In [125]:
most_expensive_muni_condo_name = df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']
most_expensive_muni_location = df_filtered.loc[df_filtered['PRICE'].idxmax()]['CITY']

least_expensive_muni_condo_name = df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']
least_expensive_muni_location = df_filtered.loc[df_filtered['PRICE'].idxmin()]['CITY']

# GENERATE STORY TEMPLATE

# STOP! DID YOU UPDATE VALUES FROM THE PREVIOUS MONTH?

In [173]:
## Calculate the increase/descrease between both months
subject_month = df_filtered["PRICE"].sum()
prior_month = sales_volume_2nd_month  # Example value, replace with actual value

# Subtract the smaller value from the larger one
if subject_month > prior_month:
    difference = subject_month - prior_month
else:
    difference = prior_month - subject_month

print(f'{difference:,.0f}')  # This will print the positive difference

47,461,057


In [216]:
story_string = f'''
\033[1mHED:\033[0m Miami-Dade condo sales volume \033[1mrises/falls\033[0m in {previous_month_name} to ${round(df_filtered["PRICE"].sum()/1_000_000)}M 
\033[1mDEK:\033[0m Number of closings \033[1mrose/fell\033[0m to {len(df_filtered)} from {total_condos_sold_2nd_month} in {month_before_last_name}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m Miami-Dade County {previous_month_name} Condo Sales Report 
\033[1mSEO DESCRIPTION:\033[0m Miami-Dade County’s condo sales volume \033[1mrises/falls\033[0m to ${round(df_filtered["PRICE"].sum()/1_000_000)} million in {previous_month_name}.
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m Miami-Dade County had ${round(df_filtered["PRICE"].sum()/1_000_000)} million in condo sales in {previous_month_name}, \033[1mup/down\033[0m from ${round(sales_volume_2nd_month/1_000_000)} million from {month_before_last_name}, @afarence reports 
ART: 

Condos against a backdrop of Miami-Dade County with an arrow pointing up 

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, Miami-Dade County, monthly condo sales, condos, condo market, {most_expensive_muni_condo_name.title()}, {most_expensive_muni_location}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m


Miami-Dade County’s NEWS PEG HERE.

{previous_month_name} condo sales totaled ${df_filtered["PRICE"].sum():,.0f} million, \033[1mup/down\033[0m from ${sales_volume_2nd_month} million in {month_before_last_name}, with a difference of ${difference:,.0f} Brokers closed {len(df_filtered)} sales last month, down from {total_condos_sold_2nd_month} sales in {month_before_last_name} and XXX in XXX, MLS data from Redfin show.

{previous_month_name} sale prices ranged from ${df_filtered["PRICE"].min():,.0f} to ${df_filtered["PRICE"].max()/1_000_000} million, with a median sale price of ${df_filtered["PRICE"].median():,.0f}. The price per square foot ranged between ${df_filtered["$/SQUARE FEET"].min():,.0f} to ${df_filtered["$/SQUARE FEET"].max():,.0f}, with a median of ${df_filtered["$/SQUARE FEET"].median():,.0f} per square foot.

In {month_before_last_name}, sales closed with a median price of ${median_sales_price_2nd_month:,}, and ${median_psf_2nd_month} per square foot.

A ${df_filtered["PRICE"].max()/1_000_000} million closing at {most_expensive_muni_condo_name.title()} took the title of priciest sale last month. {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} for ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']} per square foot after XX days on the market. XXX with XXX had the listing, and XXX with XXX represented the buyer.

The sale price was \033[1mmore/less\033[0m than {month_before_last_name}'s priciest sale. WHERE XXXX 

In contrast, {previous_month_name}'s cheapest sale was an {least_expensive_muni_condo_name} in {least_expensive_muni_location}, at {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']}. Unit XXX traded for ${df_filtered['PRICE'].min()} — or ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} per square foot — after XXX days on the market. XXX with XXX handled both sides of the deal.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="800" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of {previous_month_name}’s notable condo sales:
'''

In [217]:
print(story_string)

print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")


HED: Miami-Dade condo sales volume rises/falls in December to $457M 
DEK: Number of closings rose/fell to 677 from 672 in November
FEATURED HED:
SEO HED: Miami-Dade County December Condo Sales Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales volume rises/falls to $457 million in December.
AUTHOR: Adam Farence
RESEARCH: 
Social: Miami-Dade County had $457 million in condo sales in December, up/down from $504 million from November, @afarence reports 
ART: 

Condos against a backdrop of Miami-Dade County with an arrow pointing up 

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, monthly condo sales, condos, condo market, Porsche Design Tower, Sunny Isles Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:


Miami-Dade County’s NEWS PEG HERE.

December condo sales totaled $456,538,943 million, up/down from $504000000 million in November,